**Modelo Sinapsia**

In [1]:
'''
Install:
- pandas
- numpy
- ssqueezepy
- mne
- scipy
- sklearn
- tensorflow
'''

'\nInstall:\n- pandas\n- numpy\n- ssqueezepy\n- mne\n- scipy\n- sklearn\n- tensorflow\n'

In [1]:
import pandas as pd
import glob
import ssqueezepy #para convertir la señal en imagen
from ssqueezepy import cwt
from ssqueezepy.visuals import plot, imshow
import os
import mne #eeg analysis library
import scipy.io
#import torch.nn as nn
#import torch
import numpy as np
import re
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Ordenando el dataset (no correr)**

----------> Juntamos los archivos

In [ ]:
#subo el archivo con las ids y diagnósticos (y)
metadata = pd.read_csv('metadata_guineabissau.csv')
metadata.head(5)

In [ ]:
metadata = metadata.drop('Remarks', axis=1)
metadata = metadata.drop('recordedPeriod', axis=1)
metadata = metadata.drop('startTime', axis=1)
metadata = metadata.drop('Eyes.condition', axis=1)

metadata.head(5)

In [1]:
path = r'/content/eeg_data'
all_files = glob.glob(path + "/*.csv")

all_files = sorted(all_files)


def add_id_y (all_files, metadata):
  index = 0

  modified_dataframes_list = []
  non_eeg_columns = []

  for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    dataframe = df.iloc[:, 1:15].copy()
    dataframe['subject_id'] = metadata['subject.id'][index]
    dataframe['y'] = metadata['Group'][index]
    index += 1
    modified_dataframes_list.append(dataframe)

  return modified_dataframes_list

'''def join_csv (modified_dataframes_list):
  dataframes_list = []

  for filename in modified_dataframes_list:
    dataframe = pd.read_csv(filename, index_col=None, header=0)
    dataframes_list.append(dataframe)

  all_dataframes = pd.concat(dataframes_list, axis=0, ignore_index=True)
  all_dataframes.to_csv('eeg_data.csv', index=False)

  return all_dataframes'''

NameError: name 'glob' is not defined

In [ ]:
modified_dataframes = add_id_y(all_files, metadata)

dataframes_list = []

for dataframe in modified_dataframes:
  dataframes_list.append(dataframe)

all_dataframes = pd.concat(dataframes_list, axis=0, ignore_index=True)
all_dataframes.to_csv('eeg_data.csv', index=False)

In [ ]:
all_dataframes

**---------------Procesando el dataset para el modelo (formato mne)---------------**

**Colmna y**

Epilepsia == 1

Normal == 0

In [2]:
eeg_data = pd.read_csv('C:\\Users\\47575909\\Desktop\\eeg_new_data.csv')
eeg_data

,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,O2,P7,P8,T7,T8,subject_id,y
0,4170,-197.063158,4082,188.168421,126.526316,3798,-237.168421,3943,3957,-13.473684,-132.957895,4168,3818,-110.400000,6,control
1,4171,-192.957895,4082,188.684211,126.010526,3803,-240.768421,3951,3957,-1.684211,-135.526316,4172,3817,-103.221053,6,control
2,4171,-193.989474,4078,190.736842,121.905263,3805,-244.357895,3951,3956,-4.757895,-135.526316,4173,3813,-102.189474,6,control
3,4169,-198.600000,4076,187.147368,120.368421,3795,-244.863158,3942,3956,-19.631579,-135.010526,4168,3814,-110.915789,6,control
4,4168,-200.652632,4075,183.042105,121.400000,3790,-244.863158,3937,3955,-28.863158,-135.526316,4166,3816,-117.063158,6,control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7288955,4138,-283.221053,4202,-341.168421,171.652632,3817,-289.484211,4013,4249,158.315789,-85.778947,4221,3835,-99.631579,638,epilepsy
7288956,4141,-276.547368,4202,-337.578947,173.189474,3817,-291.021053,4014,4249,156.778947,-86.810526,4229,3833,-94.505263,638,epilepsy
7288957,4137,-280.136842,4202,-342.189474,176.778947,3818,-294.094737,4010,4249,159.347368,-85.273684,4227,3835,-99.115789,638,epilepsy
7288958,4144,-268.863158,4205,-336.547368,180.884211,3832,-293.073684,4020,4249,169.084211,-78.094737,4232,3845,-93.473684,638,epilepsy


In [3]:
eeg_data = eeg_data.round(3)
eeg_data

,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,O2,P7,P8,T7,T8,subject_id,y
0,4170,-197.063,4082,188.168,126.526,3798,-237.168,3943,3957,-13.474,-132.958,4168,3818,-110.400,6,control
1,4171,-192.958,4082,188.684,126.011,3803,-240.768,3951,3957,-1.684,-135.526,4172,3817,-103.221,6,control
2,4171,-193.989,4078,190.737,121.905,3805,-244.358,3951,3956,-4.758,-135.526,4173,3813,-102.189,6,control
3,4169,-198.600,4076,187.147,120.368,3795,-244.863,3942,3956,-19.632,-135.011,4168,3814,-110.916,6,control
4,4168,-200.653,4075,183.042,121.400,3790,-244.863,3937,3955,-28.863,-135.526,4166,3816,-117.063,6,control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7288955,4138,-283.221,4202,-341.168,171.653,3817,-289.484,4013,4249,158.316,-85.779,4221,3835,-99.632,638,epilepsy
7288956,4141,-276.547,4202,-337.579,173.189,3817,-291.021,4014,4249,156.779,-86.811,4229,3833,-94.505,638,epilepsy
7288957,4137,-280.137,4202,-342.189,176.779,3818,-294.095,4010,4249,159.347,-85.274,4227,3835,-99.116,638,epilepsy
7288958,4144,-268.863,4205,-336.547,180.884,3832,-293.074,4020,4249,169.084,-78.095,4232,3845,-93.474,638,epilepsy


In [4]:
#paso los valores categóricos a numéricos
mapping = {'epilepsy': 1, 'control': 0}
eeg_data['y'] = eeg_data['y'].map(mapping)
eeg_data

,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,O2,P7,P8,T7,T8,subject_id,y
0,4170,-197.063,4082,188.168,126.526,3798,-237.168,3943,3957,-13.474,-132.958,4168,3818,-110.400,6,0
1,4171,-192.958,4082,188.684,126.011,3803,-240.768,3951,3957,-1.684,-135.526,4172,3817,-103.221,6,0
2,4171,-193.989,4078,190.737,121.905,3805,-244.358,3951,3956,-4.758,-135.526,4173,3813,-102.189,6,0
3,4169,-198.600,4076,187.147,120.368,3795,-244.863,3942,3956,-19.632,-135.011,4168,3814,-110.916,6,0
4,4168,-200.653,4075,183.042,121.400,3790,-244.863,3937,3955,-28.863,-135.526,4166,3816,-117.063,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7288955,4138,-283.221,4202,-341.168,171.653,3817,-289.484,4013,4249,158.316,-85.779,4221,3835,-99.632,638,1
7288956,4141,-276.547,4202,-337.579,173.189,3817,-291.021,4014,4249,156.779,-86.811,4229,3833,-94.505,638,1
7288957,4137,-280.137,4202,-342.189,176.779,3818,-294.095,4010,4249,159.347,-85.274,4227,3835,-99.116,638,1
7288958,4144,-268.863,4205,-336.547,180.884,3832,-293.074,4020,4249,169.084,-78.095,4232,3845,-93.474,638,1


In [5]:
#función para convertir los datos a formato mne
def convertDF2MNE(sub):
    info = mne.create_info(list(sub.columns), ch_types=['eeg'] * len(sub.columns), sfreq=128)
    info.set_montage('standard_1020')
    data=mne.io.RawArray(sub.T, info)
    data.set_eeg_reference()
    #data.filter(l_freq=0.1,h_freq=45)
    epochs = mne.make_fixed_length_epochs(data, duration = 3, overlap = 2)
    return epochs.get_data()

In [6]:
#agrupo las líneas que tienen el mismo id, o sea que son del mismo eeg
groups = eeg_data.groupby('subject_id')

#hago una lista con cada id de cada subject (hay 97 eeg)
grp_keys = list(groups.groups.keys())
print(grp_keys)

[6, 9, 11, 12, 40, 49, 63, 64, 66, 67, 68, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 112, 113, 114, 116, 120, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 6

In [7]:
from tqdm import tqdm

In [8]:
#listas para almacenar los datos
data_list = []

for i, grpno in tqdm(enumerate(grp_keys)):
    grp = groups.get_group(grpno).drop(['subject_id'], axis=1)
    label = int(grp['y'].unique())
    subject_id = grpno
    grp = grp.drop('y', axis=1)

    #convierto mi info a formato mne
    data = convertDF2MNE(grp)  #(trials, channels, length)

    for c, x in enumerate(data):  #loop trials
        Wx, scales = cwt(x, 'morlet')
        Wx = np.abs(Wx)
        path = os.path.join('C:\\Users\\47575909\\Desktop\\scaleogram', f'subvideo_{grpno}/')
        os.makedirs(path, exist_ok=True)
        path = os.path.join(path, f'trial_{c}.npy')  # Corrección en la construcción de la ruta
        np.save(path, Wx)

        data_entry = {
            'grpno': i,
            'label': label,
            'path': path
        }

        #guardo la info del grupo (en realidad sería el "id" del eeg), la label y la path del archivo literal del eeg que ya se procesó
        data_list.append(data_entry)

#creo un dataset de TensorFlow
dataset = tf.data.Dataset.from_generator(
    generator=lambda: (data_entry for data_entry in data_list),
    output_signature={
        'grpno': tf.TensorSpec(shape=(), dtype=tf.int32),
        'label': tf.TensorSpec(shape=(), dtype=tf.int32),
        'path': tf.TensorSpec(shape=(), dtype=tf.string)
    }
)

0it [00:00, ?it/s]

Creating RawArray with float64 data, n_channels=14, n_times=34560
    Range : 0 ... 34559 =      0.000 ...   269.992 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


C:\Users\47575909\AppData\Local\Temp\ipykernel_1720\2171158857.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(grp['y'].unique())


Not setting metadata
268 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 268 events and 384 original time points ...
0 bad epochs dropped


1it [00:14, 14.29s/it]

...

Creating RawArray with float64 data, n_channels=14, n_times=34176
    Range : 0 ... 34175 =      0.000 ...   266.992 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Not setting metadata
265 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 265 events and 384 original time points ...
0 bad epochs dropped


C:\Users\47575909\AppData\Local\Temp\ipykernel_1720\2171158857.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(grp['y'].unique())
71it [16:47, 14.18s/it]

Creating RawArray with float64 data, n_channels=14, n_times=34304
    Range : 0 ... 34303 =      0.000 ...   267.992 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Not setting metadata
266 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 266 events and 384 original time points ...
0 bad epochs dropped


C:\Users\47575909\AppData\Local\Temp\ipykernel_1720\2171158857.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(grp['y'].unique())
71it [16:52, 14.26s/it]


OSError: 1236480 requested and 0 written

In [9]:
#función para serializar un ejemplo en un TFRecord
def serialize(grpno, label, path):
    feature = {
        'grpno': tf.train.Feature(int64_list=tf.train.Int64List(value=[grpno])),
        'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
        'path': tf.train.Feature(bytes_list=tf.train.BytesList(value=[path.encode('utf-8')])),
    }
    
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    
    return example_proto.SerializeToString()

In [10]:
TFdataset = './new_tfrecord_data/'
os.makedirs(TFdataset)

#función para guardar los datos de cada grupo
def save_group_to_tfrecord(grpno, dataset):
    group_file_dataset = os.path.join(TFdataset, f'{grpno}_new_data.tfrecord')
    
    with tf.io.TFRecordWriter(group_file_dataset) as writer:
        for data_entry in dataset:
            grpno = data_entry['grpno'].numpy()
            label = data_entry['label'].numpy()
            path = data_entry['path'].numpy().decode('utf-8')
            serialized = serialize(grpno, label, path)
            writer.write(serialized)
    
    print(f'Datos del grupo {grpno} se guardó en {group_file_dataset}')

In [11]:
for grpno in grp_keys:
    group_dataset = dataset.filter(lambda x: x['grpno'] == grpno)
    save_group_to_tfrecord(grpno, group_dataset)

NameError: name 'dataset' is not defined

In [4]:
#ACA SUBO LOS ARCHIVOS DE TFRECORD ||||||||||||||||||||||||||||||||||||||||||||
TFrecord_directory = 'C:\\Users\\47575909\\Desktop\\tfrecord_data' 

#lista de archivos TFRecord en la carpeta
filenames = [os.path.join(TFrecord_directory, f) for f in os.listdir(TFrecord_directory) if f.endswith('.tfrecord')]

#Hago un dataset de TensorFlow a partir de los archivos TFRecord
dataset = tf.data.TFRecordDataset(filenames)

#función para parsear los datos de TFRecord
def parse_tfrecord_fn(example):
    feature_description = {
        'grpno': tf.io.FixedLenFeature([], tf.int64),
        'label': tf.io.FixedLenFeature([], tf.int64),
        'path': tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, feature_description)
    return example

parsed_dataset = dataset.map(parse_tfrecord_fn)

#batching para argar los datos en lotes
batch_size = 5  #puedo cambiar el tamaño (en la PC del colegio el máximo es 16, pero capáz que le tengo que poner menos)
batched_dataset = parsed_dataset.batch(batch_size)

In [2]:
#ACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
#PRUEBO CON MENOS DATOS
#ACA SUBO LOS ARCHIVOS DE TFRECORD
TFrecord_directory = 'C:\\Users\\47575909\\Desktop\\tfrecord_data_prueba' 

#lista de archivos TFRecord en la carpeta
filenames = [os.path.join(TFrecord_directory, f) for f in os.listdir(TFrecord_directory) if f.endswith('.tfrecord')]

#Hago un dataset de TensorFlow a partir de los archivos TFRecord
dataset = tf.data.TFRecordDataset(filenames)

#función para parsear los datos de TFRecord
def parse_tfrecord_fn(example):
    feature_description = {
        'grpno': tf.io.FixedLenFeature([], tf.int64),
        'label': tf.io.FixedLenFeature([], tf.int64),
        'path': tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, feature_description)
    return example

parsed_dataset = dataset.map(parse_tfrecord_fn)

#batching para argar los datos en lotes
batch_size = 5  #puedo cambiar el tamaño (en la PC del colegio el máximo es 16, pero capáz que le tengo que poner menos)
batched_dataset = parsed_dataset.batch(batch_size)

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'C:\\Users\\47575909\\Desktop\\tfrecord_data_prueba'

#### Modelo (CNN) 

In [6]:
#SEPARO EN X E Y
X = []
y = []

#itero los lotes
for batch_data in tqdm(batched_dataset):
    paths = batch_data['path'].numpy()
    labels = batch_data['label'].numpy()
    
    #cargo los scaleograms desde los archivos .npy
    batch_X = []
    for path in paths:
        spectrogram = np.load(path.decode('utf-8'))
        batch_X.append(spectrogram)
    
    #agrego los scaleograms y labels a las listas X y y
    X.extend(batch_X)
    y.extend(labels)

#convierto X e y a formato .npy
X = np.array(X)
y = np.array(y)

print('Ya se separó todo!')
print('Salió todo bien, tranquina :))')

2526it [03:35, 11.74it/s]


MemoryError: Unable to allocate 4.72 MiB for an array with shape (1236480,) and data type float32

In [6]:
X.shape

(606, 14, 230, 384)

In [11]:
# Aplanar tus datos
X = X.reshape(606, -1)

In [4]:
#NORMALIZO X en rango [-1, 1] para que el cero quede en el centro
min_val = np.min(X)
max_val = np.max(X)

X = -1 + 2 * (X - min_val) / (max_val - min_val)

#X = (X - X.min()) / (X.max() - X.min()) [0, 1]

In [13]:
X.shape

(606, 1236480)

In [5]:
#LOS GUARDO EN MI COMPUTADORA
#np.save('X.npy', X) está guardada normalizada en rango [0, 1], habría que cambiar eso
np.save('X.npy', X) #esto es para probar
np.save('y.npy', y)

In [2]:
# Cargar los arreglos X e y desde los archivos .npy (PARA CUANDO EL CÓDIGO ANTERIOR YA ESTÁ CORRIDO Y NO QUIERO VOLVERLO A CORRER :d)
#X = np.load('C:\\Users\\47575909\\Desktop\\X_1.npy')
X = np.load('C:\\Users\\47575909\\Desktop\\X_PCA.npy')
y = np.load('C:\\Users\\47575909\\Desktop\\y_PCA.npy')

In [3]:
shape = X.shape

In [4]:
shape

(29588, 1000)

In [5]:
#DIVIDO EN TRAIN, TEST Y EVALUATION
#Divido los datos de train de los de test y evaluation (70/30)
X_train, X_testVal, y_train, y_testVal = train_test_split(X, y, test_size=0.3, random_state=42)

#Divido los datos de test de los de evaluation (50/50 del 30% anterior)
X_val, X_test, y_val, y_test = train_test_split(X_testVal, y_testVal, test_size=0.5, random_state=42)

cwt.shape = (14, 230, 384)

- Alto: 230 (número de filas en tus imágenes CWT).
- Ancho: 384 (número de columnas en tus imágenes CWT).
- Canales: 14 (número de canales EEG).

In [6]:
#si es que me toma mal la imput shape seguramente es por la longitud de la grabación de cada eeg, 
# por lo que dice la info del dataset no todos los eeg tienen la exacta misma duración

In [7]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)

In [8]:
X_train.shape

(20711, 1000, 1)

In [9]:
#CREO LA CNN
# Definir una capa de entrada compatible con los datos después de PCA
#input_layer = tf.keras.layers.Input(shape=(1000,))

from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

# Resto de tu modelo
model = tf.keras.models.Sequential([
    Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(1000, 1)), #le agrego un canal imaginario para que me deje la input shape
    MaxPooling1D(pool_size=2),
    
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),

    Flatten(),
    
    Dense(32, activation='relu'),
    Dropout(0.5),
    
    Dense(1, activation='sigmoid')
])

'''model = tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', input_shape = (14, 230, 384)), 
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(32, activation='relu'),

    tf.keras.layers.Dense(1, activation='sigmoid') #función sigmoide porque yo quiero que me devuelva un 1 de "si epilepsia" o un 0 de "no epilepsia"
])'''

#(LA IMPUT SHAPE LA SAQUÉ CON LO DE Wx, AGARRÉ UN EJEMPLO DE UNA PRESONA Y ME FIJÉ EL TAMAÑO PORQUE YO LE VOY A DAR INFO DE PACIENTES DE ESE FORMATO A LA CNN)

'model = tf.keras.models.Sequential([\n    \n    tf.keras.layers.Conv2D(128, (3,3), activation=\'relu\', input_shape = (14, 230, 384)), \n    tf.keras.layers.MaxPooling2D(2, 2),\n\n    tf.keras.layers.Conv2D(64, (3,3), activation=\'relu\'),\n    tf.keras.layers.MaxPooling2D(2,2),\n\n    tf.keras.layers.Dropout(0.5),\n    tf.keras.layers.Flatten(),\n\n    tf.keras.layers.Dense(32, activation=\'relu\'),\n\n    tf.keras.layers.Dense(1, activation=\'sigmoid\') #función sigmoide porque yo quiero que me devuelva un 1 de "si epilepsia" o un 0 de "no epilepsia"\n])'

In [10]:
#COMPILO
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [11]:
#ENTRENO
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

#EVALÚO
test_loss, test_acc = model.evaluate(X_val, y_val)

#capaz que debería probar con cross validation después

#ME DA PARA EL ORTO, PERO INTENTAR CON TECNICAS DE AUMENTO DE DATOS Y AGREANDAR EL DATASET D:

Epoch 1/10
648/648 [==============================] - 39s 59ms/step - loss: 0.6929 - accuracy: 0.5224 - val_loss: 0.6900 - val_accuracy: 0.5410
Epoch 2/10
648/648 [==============================] - 40s 61ms/step - loss: 0.6911 - accuracy: 0.5281 - val_loss: 0.6905 - val_accuracy: 0.5225
Epoch 3/10
648/648 [==============================] - 40s 62ms/step - loss: 0.6910 - accuracy: 0.5315 - val_loss: 0.6874 - val_accuracy: 0.5539
Epoch 4/10
648/648 [==============================] - 40s 61ms/step - loss: 0.6902 - accuracy: 0.5345 - val_loss: 0.6881 - val_accuracy: 0.5543
Epoch 5/10
648/648 [==============================] - 39s 60ms/step - loss: 0.6895 - accuracy: 0.5381 - val_loss: 0.6882 - val_accuracy: 0.5424
Epoch 6/10
648/648 [==============================] - 40s 62ms/step - loss: 0.6899 - accuracy: 0.5397 - val_loss: 0.6877 - val_accuracy: 0.5417
Epoch 7/10
648/648 [==============================] - 39s 60ms/step - loss: 0.6891 - accuracy: 0.5400 - val_loss: 0.6864 - val_accuracy:

In [12]:
#VEO LA PERFORMANCE
print("Accuracy : ", test_acc)
print("Loss : ", test_loss)

Accuracy :  0.5538530945777893
Loss :  0.687100887298584


In [13]:
max_shift = 50  # Máximo desplazamiento en muestras
X_augmented = []
y_augmented = []

for sample, label in zip(X_train, y_train):
    shift = np.random.randint(-max_shift, max_shift)
    
    augmented_sample = np.roll(sample, shift)
    X_augmented.append(augmented_sample)
    
    augmented_label = np.roll(label, -shift)  # Alinea las etiquetas con el desplazamiento
    y_augmented.append(augmented_label)

X_augmented = np.array(X_augmented)
y_augmented = np.array(y_augmented)

In [14]:
#datos generadossssssssssss
model.fit(X_augmented, y_augmented, epochs=10, validation_data=(X_val, y_val))

test_loss, test_acc = model.evaluate(X_val, y_val)

Epoch 1/10
648/648 [==============================] - 39s 61ms/step - loss: 0.6943 - accuracy: 0.5161 - val_loss: 0.6921 - val_accuracy: 0.5212
Epoch 2/10
648/648 [==============================] - 39s 60ms/step - loss: 0.6925 - accuracy: 0.5185 - val_loss: 0.6901 - val_accuracy: 0.5408
Epoch 3/10
648/648 [==============================] - 38s 59ms/step - loss: 0.6924 - accuracy: 0.5192 - val_loss: 0.6918 - val_accuracy: 0.5237
Epoch 4/10
648/648 [==============================] - 38s 59ms/step - loss: 0.6926 - accuracy: 0.5218 - val_loss: 0.6914 - val_accuracy: 0.5300
Epoch 5/10
648/648 [==============================] - 38s 59ms/step - loss: 0.6920 - accuracy: 0.5258 - val_loss: 0.6895 - val_accuracy: 0.5480
Epoch 6/10
648/648 [==============================] - 38s 59ms/step - loss: 0.6919 - accuracy: 0.5249 - val_loss: 0.6894 - val_accuracy: 0.5475
Epoch 7/10
648/648 [==============================] - 38s 59ms/step - loss: 0.6918 - accuracy: 0.5277 - val_loss: 0.6906 - val_accuracy:

In [15]:
print("Accuracy : ", test_acc)
print("Loss : ", test_loss)

Accuracy :  0.5277152061462402
Loss :  0.6904980540275574


# ACÁ EXPORTAMOS EL MODELO

In [ ]:
#guardamos el modelo
model.save('modelo_SinapsIA.h5')

Álvaro, hola, yo te diría que empecemos con el modelo casi sin entrenar para que ya sepamos como pasarlo a la web. Vos lo que vas a hacer es pedirme los archivos para entrenarlo poquito (que literalmente van a ser solo 2 eeg porque sino va a ser muy pesado, yo estoy trabajando en reducir el tamaño del dataset). Entonces, vos te tendrías que descargar este notebook y correrlo en tu compu y entrenarlo así con estos pocos datos que no te van a pesar (se supone), y ahí lo conectas con lo de Jano. Para cuando los datos ya estén bien, supongo que tendremos que hacer lo mismo pero desde la pc porque aunque le reduzca el tamaño va a ser imposible que entre en la netbook. Solamente hay que ejecutar el código que vos hagas con el mismo modelo pero entrenado con más datos.

ADEMÁS, TE HICE UN NOTEBOOK CON TODO PASADO, PERO IUAL MIRA ESTE.

In [ ]:
#investigar PCA para tensores (TAMBIÉN INVESTGAR MÁS PARA TENSORES)
#https://youtu.be/FgakZw6K1QQ?si=GrRc8tCp2eEfNKaX

#TAMBIÉN CHONA ME RECOMIENDA UNA LIBRERÍA PaCMACP para mi caso que uso tensores de tensorflow

**Tengo que hacer y notas**
##### **--> pasar todo a una pc**
##### **--> separar en datos de train, test y validation**

`78` = train

`10` = validation

`9` = test

##### **--> ver el formato y como feedearselo a una red neuronal**

Opciones de modelos a investigar:

*   `Convolutional neural network (CNN)`
*   `Recurrent neural network (RNN)`



In [ ]:
'''CHONA ME DIJO QUE USARA LOS DATOS SIN PCA Y QUE HAGA UN POOLING GRANDE AL PRINCIPIO DE LA CNN PARA ACHICAR LOS DATOS DE UNA CUANDO ENTRAN. 
ADEMÁS, TENGO QUE HACER CONV2D EN VES DE 1D Y TENGO QUE AGREGARLE LOS NUEVOS DATOS. ADEMÁS TENGO QUE USAR EL PRETRAINED MODEL RESNET QUE USO 
EL INDIO EN EL VIDEO PARA TENER YO YA HECHO EL COSO Y CON LOS WEIGHTS Y QUE NO ME TRADE TANTO EL ENTRENAMIENTO. YO LE CAMBIARÍA LO DEL POOL AL 
PRINCIPIO Y LO ADAPTARÍA AL FINAL DE LA RED PARA QUE SE AJUSTE A MIS DATOS.
- TENGO QUE CORRER EL CODIGO PARA EL DATASET DE NUEVO PERO SIN EL PCA 
- VOLVER A ARREGLAR LA CNN PARA QUE ADMITA LA VIEJA INPUT SHAPE
- USAR UN MODELO RESNET PREENTRENADO CON DATOS COHERENTES (PREFERENTEMENTE EL QUE USÓ EL INDIO)
- INCORPORAR LO QUE ME DIJO CHONA DE CONV2D Y UN POOLING PARA QUE SE ADAPTE A RESNET (AdaptiveAvgPool2d - https://stackoverflow.com/questions/61955991/change-input-shape-dimensions-for-resnet-model-pytorch)

CODIGOS DEL INDIO:
https://github.com/talhaanwarch/youtube-tutorials/blob/main/eeg-conv2d.ipynb
https://github.com/talhaanwarch/youtube-tutorials/blob/main/eeg_epilepsy.ipynb
'''